In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}
.uplot{
   background-color: white;
}
</style>

In [3]:
import sys

sys.path.insert(0, "D:/my_repo/simple-quant/anywidget_demo/src")
sys.path.insert(0, "D:/my_repo/simple-quant/simple_quant")

In [4]:
from get_data import get_data, work_day, work_time, get_or_read_data
import datetime
import json

demo_time = datetime.datetime.now()
# demo_time = datetime.datetime(2024, 3, 15, 14, 0)
d = work_day(demo_time)
t = work_time(demo_time)
print(demo_time, d, t)

2024-04-03 18:07:20.802055 True False


In [5]:
# get_data
import pandas as pd
from pathlib import Path

ohlc_config = {
    "name": "SHFE.rb2405",
    # "name": "CZCE.MA405",
    # "name": "DCE.m2405",
    # "name": "CZCE.RM405",
    # "name": "DCE.c2405",
    # "name": "DCE.l2405",
    # "name": "CZCE.TA405",
    "frequency": "300s",  # 15s, 30s, 60s, 300s, 900s, 1800s
    "count": 3000,
    "refresh": True,
    # "end_time": None,
    "end_time": "2024-03-22 23:00:00",
}

path = Path(
    f"../../csv/{ohlc_config['name']}/{ohlc_config['name']} {ohlc_config['frequency']} {ohlc_config['count']} {ohlc_config['end_time'].replace(':','_')}.csv"
)
path.parent.mkdir(parents=True, exist_ok=True)

ohlc_data = get_or_read_data(ohlc_config, path)
ohlc_data

,time,open,high,low,close,volume,date
0,1.705284e+09,3902.0,3906.0,3899.0,3905.0,10216,2024-01-15 10:00:00+08:00
1,1.705284e+09,3906.0,3907.0,3903.0,3904.0,5533,2024-01-15 10:05:00+08:00
2,1.705285e+09,3904.0,3907.0,3900.0,3900.0,8550,2024-01-15 10:10:00+08:00
3,1.705286e+09,3900.0,3902.0,3896.0,3896.0,10191,2024-01-15 10:30:00+08:00
4,1.705286e+09,3896.0,3902.0,3895.0,3900.0,9058,2024-01-15 10:35:00+08:00
...,...,...,...,...,...,...,...
2995,1.711118e+09,3610.0,3611.0,3605.0,3609.0,14903,2024-03-22 22:35:00+08:00
2996,1.711118e+09,3609.0,3612.0,3608.0,3611.0,11282,2024-03-22 22:40:00+08:00
2997,1.711119e+09,3611.0,3619.0,3609.0,3610.0,33380,2024-03-22 22:45:00+08:00
2998,1.711119e+09,3610.0,3611.0,3608.0,3610.0,14482,2024-03-22 22:50:00+08:00


In [6]:
# run_add_ta
from add_ta import add_ta


def run_add_ta(
    ohlc_data,
    ta_config={"ma_list": [4, 8, 16], "ma_name": "sma"},
):
    if "ma_name" in ta_config:
        ta_config["ma"] = [f"{ta_config['ma_name']}_{i}" for i in ta_config["ma_list"]]
    if "bbands_length" in ta_config:
        ta_config["bbands"] = [
            f"{i}_{ta_config['bbands_length']}_{ta_config['bbands_std']}"
            for i in ["BBL", "BBU"]
        ]
    if "atr_length" in ta_config:
        ta_config["atr"] = "atr"

    ohlc_data = add_ta(ohlc_data, ta_config)
    return ohlc_data


ta_config = {
    "ma_list": [4, 32, 64, 128],
    "ma_name": "sma",
    "bbands_length": 128,
    "bbands_std": 2,
    "rsi_length": 14,
    "atr_length": 14,
}

ta_data = run_add_ta(ohlc_data, ta_config=ta_config)
ta_data

,sma_4,sma_32,sma_64,sma_128,BBL_128_2,BBU_128_2,rsi,atr
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3901.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3900.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2995,3608.25,3612.22,3610.73,3609.71,3586.14,3633.28,45.62,7.15
2996,3609.00,3612.38,3610.61,3609.96,3587.03,3632.90,48.07,6.92
2997,3609.75,3612.53,3610.47,3610.20,3587.94,3632.47,46.93,7.14
2998,3610.00,3612.62,3610.38,3610.42,3588.72,3632.12,46.93,6.85


In [7]:
# run_signals
import pandas_ta as ta


def run_signals(ohlc_data, ta_data, ta_config):
    close = ohlc_data["close"]
    ma_a = ta_data[ta_config["ma"][0]]
    ma_b = ta_data[ta_config["ma"][1]]
    gc_data = ta.tsignals(
        (close > ma_a) & (ma_a < ma_b),
        asbool=True,
    )
    dc_data = ta.tsignals(
        (close < ma_a) & (ma_a > ma_b),
        asbool=True,
    )
    return gc_data, dc_data


gc_data, dc_data = run_signals(ohlc_data, ta_data, ta_config)
gc_data

,TS_Trends,TS_Trades,TS_Entries,TS_Exits
0,False,0,False,False
1,False,0,False,False
2,False,0,False,False
3,False,0,False,False
4,False,0,False,False
...,...,...,...,...
2995,True,0,False,False
2996,True,0,False,False
2997,True,0,False,False
2998,False,-1,False,True


In [8]:
# run_backtest
from backtest import backtest, analysis_data

bt_config = {"mode": "sl", "wait": 20, "stop": 10}
bt_config = {"mode": "tsl", "stop": 10}
bt_config = {"mode": "tsl_atr", "rate": 1.5}
bt_config = {"mode": "tsl2", "wait": 20, "stop": 15}


def run_backtest(ohlc_data, ta_data, gc, dc, mode):
    # mode: sl, tsl, tsl2, tsl_atr

    backtest(
        gc,
        ohlc_data,
        ta_data,
        way="up",
        mode=mode["mode"],
        wait=mode.get("wait", 10),
        stop=mode.get("stop", 10),
        rate=mode.get("rate", 1),
    )
    backtest(
        dc,
        ohlc_data,
        ta_data,
        way="down",
        mode=mode["mode"],
        wait=mode.get("wait", 10),
        stop=mode.get("stop", 10),
        rate=mode.get("rate", 1),
    )
    return gc, dc


gc_data, dc_data = run_backtest(ohlc_data, ta_data, gc_data, dc_data, bt_config)
gc_data

,TS_Trends,TS_Trades,TS_Entries,TS_Exits,Entries,Exits
0,False,0,False,False,False,False
1,False,0,False,False,False,False
2,False,0,False,False,False,False
3,False,0,False,False,False,False
4,False,0,False,False,False,False
...,...,...,...,...,...,...
2995,True,0,False,False,False,False
2996,True,0,False,False,False,False
2997,True,0,False,False,False,False
2998,False,-1,False,True,False,False


In [9]:
# run_analysis_data
def run_analysis_data(gc, dc):

    storeDataUp = analysis_data(
        ohlc_data, gc[gc.columns[-2]], gc[gc.columns[-1]], mode="up"
    )
    storeDataDown = analysis_data(
        ohlc_data, dc[dc.columns[-2]], dc[dc.columns[-1]], mode="down"
    )
    store_data = storeDataUp.join(storeDataDown, lsuffix="_up", rsuffix="_down")
    store_data.drop(["time_down"], axis=1, inplace=True)
    store_data.rename(columns={"time_up": "time"}, inplace=True)
    store_data["money"] = store_data["money_up"] + store_data["money_down"]
    return store_data


store_data = run_analysis_data(gc_data, dc_data)
store_data

,time,money_up,money_cur_up,success_up,failure_up,money_down,money_cur_down,success_down,failure_down,money
0,1.705284e+09,0.0,0.0,False,False,0.0,0.0,False,False,0.0
1,1.705284e+09,0.0,0.0,False,False,0.0,0.0,False,False,0.0
2,1.705285e+09,0.0,0.0,False,False,0.0,0.0,False,False,0.0
3,1.705286e+09,0.0,0.0,False,False,0.0,0.0,False,False,0.0
4,1.705286e+09,0.0,0.0,False,False,0.0,0.0,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...
2995,1.711118e+09,-142.0,-142.0,False,False,275.0,286.0,False,False,133.0
2996,1.711118e+09,-142.0,-140.0,False,False,275.0,284.0,False,False,133.0
2997,1.711119e+09,-142.0,-141.0,False,False,275.0,285.0,False,False,133.0
2998,1.711119e+09,-142.0,-141.0,False,False,275.0,285.0,False,False,133.0


In [10]:
# run_anywidget


from anywidget_demo import Counter


def run_anywidget(
    ohlc_data, ta_data, gc_data, dc_data, store_data, ohlc_config, ta_config, bt_config, boardIdx="demo",showArr=[True,True,True]
):

    c = Counter()

    c.value = 0  # 暂时没用
    c.ohlcData = json.dumps(ohlc_data.fillna(value="NaN").to_dict(orient="list"))
    c.taData = json.dumps(ta_data.fillna(value="NaN").to_dict(orient="list"))
    c.gcData = json.dumps(gc_data.fillna(value="NaN").to_dict(orient="list"))
    c.dcData = json.dumps(dc_data.fillna(value="NaN").to_dict(orient="list"))
    c.storeData = json.dumps(store_data.fillna(value="NaN").to_dict(orient="list"))
    c.ohlcConfig = json.dumps(ohlc_config)
    c.taConfig = json.dumps(ta_config)
    c.btConfig = json.dumps(bt_config)
    c.chartOpt = json.dumps(
        {
            "width": "70%",
            "height": "35%",
            "scales": {"x": {"time": True, "distr": 2}, "y": {}},
            "legend": {
                "show": False,
            },
            "axes": [{"size": 1}],
            "show": showArr[0],
        }
    )

    c.subOpt = json.dumps(
        {
            "width": "70%",
            "height": "13%",
            "scales": {"x": {"time": True, "distr": 2}, "y": {}},
            "legend": {
                "show": False,
            },
            "axes": [{"size": 1}],
            "show": showArr[1],
        }
    )

    c.storeOpt = json.dumps(
        {
            "width": "70%",
            "height": "20%",
            "scales": {"x": {"time": True, "distr": 2}, "y": {}},
            "legend": {
                "show": False,
            },
            "show": showArr[2],
        }
    )

    c.pluginOpt = json.dumps(
        {
            "factorWheel": 0.3,
            "factorPan": 0.15,
            "panButton": 0,
            "scrollZoom": True,
            "syncZoomPan": True,
            "boardIdx":boardIdx
        }
    )  # panButton: None关闭, 0鼠标左键,1中键,2右键,3后退键,4前进键

    # c.storeData = "0,1,2,3,4\n1,2,3,4,5\n2,3,4,5,6,\n"

    return c

In [11]:
# run_anywidget(
#     ohlc_data, ta_data, gc_data, dc_data, store_data, ohlc_config, ta_config, bt_config, boardIdx="test", showArr=[True,True,True]
# )

In [12]:
# optimize
def run_signals(ohlc_data, ta_data, ta_config):
    close = ohlc_data["close"]
    ma_a = ta_data[ta_config["ma"][0]]
    ma_b = ta_data[ta_config["ma"][1]]
    ma_c = ta_data[ta_config["ma"][2]]
    ma_d = ta_data[ta_config["ma"][3]]
    gc_data = ta.tsignals(
        (ma_c>ma_d)
        & (close> ma_c)
        & (close< ma_b)
        & (close>ma_a)
        ,
        asbool=True,
    )
    dc_data = ta.tsignals(
        (ma_c<ma_d)
        & (close< ma_c)
        & (close> ma_b)
        & (close<ma_a)
        ,
        asbool=True,
    )
    return gc_data, dc_data


bt_config = {"mode": "sl", "wait": 20, "stop": 10}
bt_config = {"mode": "tsl", "stop": 10}
bt_config = {"mode": "tsl_atr", "rate": 1.5}
bt_config = {"mode": "tsl2", "wait": 20, "stop": 15}

plotArr = []


def optimize():
    for idx,config in enumerate([{"ma_list": [4, 32, 64, 128]}]):
        ta_config["ma_list"] = config["ma_list"]
        ta_data = run_add_ta(ohlc_data, ta_config=ta_config)
        gc_data, dc_data = run_signals(ohlc_data, ta_data, ta_config)
        gc_data, dc_data = run_backtest(ohlc_data, ta_data, gc_data, dc_data, bt_config)
        store_data = run_analysis_data(gc_data, dc_data)
        plotArr.append(
            run_anywidget(
                ohlc_data,
                ta_data,
                gc_data,
                dc_data,
                store_data,
                ohlc_config,
                ta_config,
                bt_config,
                boardIdx=idx,
                showArr=[True, True, True],
            )
        )

optimize()

这个策略, 只有螺纹是赚的(300s), 其他都亏

In [13]:
plotArr[0]

Counter(btConfig='{"mode": "tsl2", "wait": 20, "stop": 15}', chartOpt='{"width": "70%", "height": "35%", "scal…